In [1]:
import osmium
import sys
from neo4j import GraphDatabase, basic_auth
from copy import deepcopy

In [2]:
url = 'http://download.geofabrik.de/europe/bulgaria-latest.osm.pbf'

In [3]:
class GraphNode:

    def __init__(self, node_id, lat, lon):
        self.node_id = node_id
        self.lat = lat
        self.lon = lon

In [4]:
class GraphEdge:

    def __init__(self, start_node_id, end_node_id, distance, rating):
        self.start_node_id = start_node_id
        self.end_node_id = end_node_id
        self.distance = distance
        self.rating = rating

In [5]:
class Neo4jHandler():
    
    def __init__(self):
        self.driver = GraphDatabase.driver(
            "bolt://54.167.190.221:7687",
            auth=basic_auth("neo4j", "release-shows-breads"))
        self.nodes = []
        self.edges = []
        
    def add_node(self, node):
        self.nodes.append(node)
        print(node.node_id)

    def add_edge(self, edge):
        self.edges.append(edge)

    def create_objects(self):
        with self.driver.session() as session:
            session.write_transaction(self.create_nodes)
    
    def close(self):
        self.driver.close()
            
    def create_nodes(self, transaction):
        for node in self.nodes:
            query = self.build_create_node_query(node)
            transaction.run("""
                CREATE (n)
                SET node_id = $node_id
                SET lat = $lat
                SET lon = $lon
                """, node_id=node.node_id, lat=node.lat, lon=node.lon)

In [6]:
class FileStatsHandler(osmium.SimpleHandler):

    def __init__(self, neoHandler):
        super(FileStatsHandler, self).__init__()
        self.ways = []
        self.rels = 0
        self.velos = []
        self.neoHandler = neoHandler

    def node(self, n):
        id = deepcopy(n.id)
        lat, lon = deepcopy(str(n.location)).split('/')
        node = GraphNode(n.id, lat, lon)
        self.neoHandler.add_node(node)

    def way(self, w):
        self.ways += 1

    def relation(self, r):
        self.rels += 1


In [7]:
neoHandler = Neo4jHandler()
h = FileStatsHandler(neoHandler)

h.apply_file(url, locations=True, idx='flex_mem')

neoHandler.create_objects()
neoHandler.close()

print(f'Nodes: {h.nodes}')
# print(f'Ways: {h.ways}')
# print(f'Relations: {h.rels}')

7
1046781741
1046782076
1046782138
1046782169
1046782267
1046782549
1046782721
1046782744
1046782893
1046782903
1046782914
1046782930
1046783047
1046783098
1046783356
1046783359
1046783383
1046783392
1046783398
1046783403
1046783405
1046783410
1046783432
1046783433
1046783477
1046783486
1046783494
1046783514
1046783517
1046783519
1046783529
1046783534
1046783537
1046783541
1046783544
1046783558
1046783567
1046783590
1046783650
1046783684
1046783692
1046783729
1046784014
1046784060
1046784080
1046784085
1046784182
1046784400
1046784476
1046784528
1046784571
1046784585
1046784651
1046784763
1046784895
1046785081
1046785092
1046785096
1046785099
1046785112
1046785122
1046785124
1046785140
1046785142
1046785143
1046785145
1046785149
1046785150
1046785161
1046785162
1046785283
1046785311
1046785317
1046785331
1046785336
1046785344
1046785392
1046785434
1046785436
1046785446
1046785460
1046785473
1046785475
1046785479
1046785504
1046785505
1046785517
1046785523
1046785525
1046785526
10467855

KeyboardInterrupt: 

In [7]:
class Convert(osmium.SimpleHandler):
    def __init__(self, writer):
        super(Convert, self).__init__()
        self.writer = writer

    def node(self, n):
        self.writer.add_node(n)

    def way(self, w):
        self.writer.add_way(w)

    def relation(self, r):
        self.writer.add_relation(r)

In [8]:
in_file = 'bulgaria-latest.osm.pbf'
out_file = 'monaco.osm'

In [9]:
!rm {out_file}

In [10]:
writer = osmium.SimpleWriter(out_file)
handler = Convert(writer)
handler.apply_file(url)
writer.close()